In [1]:
import numpy as np
import pandas as pd

# Nonuniform flow

In [2]:
def nonuniform(Ls, Zip, dx):
    q = 10.0
    n = 0.03
    ib = 1/400
    g = 9.8
    
    h0 = (q**2*n**2/ib)**0.3 #等流水深
    hc = (q**2/g)**(1/3) # 限界水深
    
    h = np.zeros_like(Ls) #水深の配列
    zb = Zip.copy()
    
    h[0] = h0 #下流端条件
    for i in range(1,len(h)):
        h[i] = h[i-1] #収束計算の初期値：一つ下流側の断面の水深
        f = 1.0 #仮値
        dfdh = 1.0 #仮値
        while np.abs(f/dfdh) > 10**(-8): # 反復計算の収束条件
            f = q**2/2.0/g/h[i]**2 + h[i] + zb[i] \
              -(q**2/2.0/g/h[i-1]**2 + h[i-1] + zb[i-1]) \
              - 0.5*(q**2*n**2/h[i]**(10/3) + q**2*n**2/h[i-1]**(10/3))*dx
            dfdh = -q**2/g/h[i]**3 + 1 + 5/3*q**2*n**2/h[i]**(13/3)*dx
            h[i] -= f/dfdh
            
    return h

In [20]:
df = pd.read_csv('zb2.csv',index_col=0)

# import requests
# import io

# url = 'https://raw.githubusercontent.com/computational-sediment-hyd/1DSWEwithSmallerDx/main/zb2.csv'
# response = requests.get(url, timeout=10, verify=False)
# df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), index_col=0)

In [17]:
dx = 0.5
Ls1 = np.arange(0,10000.1,dx)
Zip1 = np.interp(Ls1, df.L, df.Z)

hc1 = nonuniform(Ls1, Zip1, dx)

dx = 10.0
Ls2 = np.arange(0,10000.1, dx)
Zip2 = np.interp(Ls2, df.L, df.Z)

hc2 = nonuniform(Ls2, Zip2, dx)

dx = 100.0
Ls3 = np.arange(0,10000.1, dx)
Zip3 = np.interp(Ls3, df.L, df.Z)

hc3 = nonuniform(Ls3, Zip3, dx)

 - export

In [19]:
dic = {"0.5":hc1, "10":hc2, "100":hc3}
d = np.savez("NonUniform", **dic)

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
fig = hv.Curve((df.L,df.Z), label='Zb dx=0.5m').options(show_grid=True, ylabel='elevation[m]', xlabel='distance[m]' ,color='k') \
* hv.Scatter((Ls2, Zip2), label='Zb dx=10m').options(color='r',size=6) \
* hv.Scatter((Ls3, Zip3), label='Zb dx=100m').options(color='g',size=6) \
* hv.Curve((Ls1, Zip1 + hc1), label='dx=0.5m' ).options(color='b') \
* hv.Curve((Ls2, Zip2 + hc2), label='dx=10m').options(color='r') \
* hv.Curve((Ls3, Zip3 + hc3), label='dx=100m').options(color='g')

In [ ]:
hv.extension('bokeh')
figo = fig.options(width=500, legend_position='bottom_right')
# d = hv.save(figo, 'NUdx.html')
figo 

In [ ]:
hv.extension('bokeh')
figo2 = fig.options(xlim=(3050, 3350), ylim=(7, 12), width=700, height=600, legend_position='right')
# d = hv.save(figo2, 'NUdx2.html')
figo2